# Notebook du micro-projet
## But du dataset:
Le fichier est disponible sur le git avec le nom "dataset.py" : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier récupère et prépare les données pour qu'elles soit exploité par le réseau de neurone.

## Déroulement du code :

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import random

Commençons par bon nombre d'import. Principalement lié à Keras et Tensorflow (dont Keras est une sur-couche).
Jusque là  en se qui concerne le code, tout va bien.

On va procéder a un formatage du jeu de données : on commence alors par aller chercher les différentes images dans images.npy. Ensuite on va assigner pour chaque image, le bon label (les labels fournis de base n'étant pas bien assigné). C'est ce qui va permettre d'entrainer le réseau et verifier qu'il a bien appris. Puis on va modifier le format des images avant de les passer dans le modele afin de faciliter tous les calculs/comparaison. Afin d'améliorer l'entrainement, on va mélanger le dataset, cela va permettre que le réseau ne traite pas toutes les valeurs de 9 puis toutes les valeurs de 0 etc. Cela améliorer les performances lors de l'entrainement, on fini en séparant en 80% des données pour l'entraînement et 20% pour le test.

In [39]:
def load_data():
    # Load images
    images = np.load('datasets/images.npy')
    classes = np.zeros(images.shape[0])
    classes[:204] = 9
    classes[204:409] = 0
    classes[409:615] = 7
    classes[615:822] = 6
    classes[822:1028] = 1
    classes[1028:1236] = 8
    classes[1236:1443] = 4
    classes[1443:1649] = 3
    classes[1649:1855] = 2
    classes[1855:] = 5
    
    classes = np_utils.to_categorical(classes, 10)
    images = images.reshape(images.shape[0], 64, 64, 1)
    
    combined = list(zip(images, classes))
    random.shuffle(combined)
    images[:], classes[:] = zip(*combined)
    
    splitIndex = int(len(images) * 0.8)
    x_train = images[:splitIndex]
    x_test = images[splitIndex:]
    y_train = classes[:splitIndex]
    y_test = classes[splitIndex:]
    
    return (x_train, y_train), (x_test, y_test)

## But du modèle :
Ce fichier est disponible sur le git avec le nom model.py : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier centralise une grande partie de se projet et en particulier le model qui va réunir les différentes couches du réseau.

## Déroulement du code :

Dans cette partie, on va comparer les résultats d'un modèle classique avec ceux d'un modèle convolutionnel. 

### Modèle classique :

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data()

model2 = Sequential()
model2.add(Dense(512, activation='relu', input_shape=(64,64,1)))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(
        x_train,
        y_train,
        epochs=30,
        validation_data=(x_test, y_test),
        verbose=1
)

Train on 1649 samples, validate on 413 samples
Epoch 1/15


### Modèle convolutionnel :

In [57]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(64, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(128, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(256, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

**Commentaire sur le model**
On applique un modele convolutionnel classique: on alterne des couches de convolutions avec un filtre (2,2), une fonction d'activation Relu, et le data_format 'channels_last' qui est  à mettre en corrélation avec le formatage des données effectué à l'étape précéde,t. On alterne donc ces couches convolutionnelles avec de couches de max-pooling avec toujours un filtre (2,2) : cela permet de faire du sous-échantillonage et ainsi permettre au réseau d'extraire des features à différents niveau d'abstractions. On rajoute une couche flatten permettant de faire la liaisons entre les couches convolutionneles et les couches denses, les couches denses permettent d'obtenir le résultat final, la fonction softmax permet d'interpreter le résultat comme une probabilité. On a également ajouter deux couches de Dropout qui permettent de désactiver de manière aléaoire des sorties de neurones, cela nous as permis d'améliorer nos résultats.

Ensuite on compile ce model, pour ce faire on a 3 paramètres, en particulier l'optimizer (adam, optimiseur stochastique) et le loss (crossentropy car ce ne sont pas de simple entier) :

In [58]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Il y a ensuite un load. Il appel une méthode de l'autre fichier python (DataSet).

In [59]:
(x_train, y_train), (x_test, y_test) = load_data()

Ici nous avons 4 paramètres d'initialiser :
- Epochs : Le nombre d'époque utiliser. Plus cette valeur est élever plus le réseau sera efficace. Mais plus le code sera long à  exécuter.
- Batch_size : La taille des différents lots.
- Data_augmentation : Comme son nom l'indique, ce booléen indique si on utilise la data augmentation (multiplié le nombre de données)

In [60]:
EPOCHS = 50
BATCH_SIZE = 32
DATA_AUGMENTATION = True

Ensuite, on va permettre l'augmentation de données, ce qui nous as permis de comparer les résultats avec/sans cette augmentation. L'augmentation de données fonctionne de la manière suivante : on va effectuer sur les images du dataset des transformations diverses (rotation, scaling...), cela permet d'avoir beaucoup plus de données et d'avoir un réseau de neurones qui apprendra a associer le bon label a une image par exemple un peu de travers ou autre. 
Nous avons réalisé une petite démonstration web à l'aide de TensorFlowJS, nous avons constaté de bien meilleur résultat avec l'augmentation de données.

In [61]:
if DATA_AUGMENTATION:
    # With data augmentation
    generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=[0.7, 1.3],
    )
    model.fit_generator(
        generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(x_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=generator.flow(x_test, y_test, batch_size=BATCH_SIZE)
    )
else:
    # Without data augmentation
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_test, y_test)
)

Epoch 1/50


C:\Users\colin\Anaconda3\lib\site-packages\keras_preprocessing\image.py:988: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\colin\Anaconda3\lib\site-packages\keras_preprocessing\image.py:996: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 6/51 [==>...........................] - ETA: 3:29 - loss: 2.2284 - acc: 0.1302

KeyboardInterrupt: 

Le générateur est un ImageDataGenerator, générateur intégrer à Keras. Ici on a pris quelques décision pour l'augmentation tel que la rotation des images et l'intervalle de zoom. On utilise pour finir le model créer précédemment pour entraîné le réseau.
Pour finir on va tester l'efficacité du réseau de neurone grâce à la méthode evaluate du model.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

## Caractérisation du cout de calcul :

BLABLABLA